In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime 
import seaborn as sb
import scipy.stats as stat
%matplotlib inline
import random
from geopy.distance import vincenty

In [2]:
bikeshare = pd.read_csv('bikeshare_reduced.csv', 
                        parse_dates=['Start date', 'End date', 'start_date_short', 'end_date_short'])
bikeshare.drop('Unnamed: 0', 1, inplace=True)

lat_lon = pd.read_csv('bikeshare_locations.csv')
print(lat_lon.head())

    OBJECTID  ID                                     ADDRESS  TERMINAL_NUMBER  \
0  173869668   1                         Eads St & 15th St S            31000   
1  173869669   2                             18th & Eads St.            31001   
2  173869670   3                           20th & Crystal Dr            31002   
3  173869671   4                           15th & Crystal Dr            31003   
4  173869672   5  Aurora Hills Community Ctr/18th & Hayes St            31004   

    LATITUDE  LONGITUDE INSTALLED LOCKED  INSTALL_DATE  REMOVAL_DATE  \
0  38.858971 -77.053230       YES     NO           NaN           NaN   
1  38.857250 -77.053320       YES     NO           NaN           NaN   
2  38.856425 -77.049232       YES     NO           NaN           NaN   
3  38.860170 -77.049593       YES     NO           NaN           NaN   
4  38.857866 -77.059490       YES     NO           NaN           NaN   

  TEMPORARY_INSTALL  NUMBER_OF_BIKES  NUMBER_OF_EMPTY_DOCKS            X  \
0   

In [3]:
lat_lon = lat_lon[['ADDRESS', 'LATITUDE', 'LONGITUDE']]

In [4]:
lat_lon.head()

,ADDRESS,LATITUDE,LONGITUDE
0,Eads St & 15th St S,38.858971,-77.053230
1,18th & Eads St.,38.857250,-77.053320
2,20th & Crystal Dr,38.856425,-77.049232
3,15th & Crystal Dr,38.860170,-77.049593
4,Aurora Hills Community Ctr/18th & Hayes St,38.857866,-77.059490


In [5]:
bikeshare.head()

,Start date,End date,Start station,End station,Member Type,start_date_short,end_date_short,time_diff,season,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed
0,2011-06-30 23:59:00,2011-07-01 00:09:00,11th & Kenyon St NW (31102),Calvert St & Woodley Pl NW (31106),Registered,2011-06-30,2011-07-01,10.0,3,6,0,4,1,1,0.696667,0.637008,0.434167,0.185312
1,2011-06-30 23:58:00,2011-07-01 00:03:00,Lamont & Mt Pleasant NW (31107),Adams Mill & Columbia Rd NW (31104),Registered,2011-06-30,2011-07-01,5.0,3,6,0,4,1,1,0.696667,0.637008,0.434167,0.185312
2,2011-06-30 23:59:00,2011-07-01 00:09:00,Massachusetts Ave & Dupont Circle NW (31200),Adams Mill & Columbia Rd NW (31104),Casual,2011-06-30,2011-07-01,10.0,3,6,0,4,1,1,0.696667,0.637008,0.434167,0.185312
3,2011-06-30 23:58:00,2011-07-01 00:17:00,4th St & Massachusetts Ave NW (31604),C & O Canal & Wisconsin Ave NW (31225),Registered,2011-06-30,2011-07-01,19.0,3,6,0,4,1,1,0.696667,0.637008,0.434167,0.185312
4,2011-06-30 23:58:00,2011-07-01 00:28:00,10th St & Constitution Ave NW (31219),14th & D St NW / Ronald Reagan Building (31231),Casual,2011-06-30,2011-07-01,30.0,3,6,0,4,1,1,0.696667,0.637008,0.434167,0.185312


In [6]:
small = bikeshare.copy()

In [7]:
small['start_station'], small['start_station_code'] = small['Start station'].str.split("(", 1).str

In [8]:
small['start_station_code'] = small['start_station_code'].str.replace(')', '')

In [9]:
small['start_station'] = small['start_station'].str.strip()

In [10]:
sum(small['start_station_code'].isnull())

354030

In [11]:
len(small['Start station'].unique())

294

In [12]:
len(lat_lon)

485

In [13]:
small['end_station'], small['end_station_code'] = small['End station'].str.split("(", 1).str

In [14]:
small['end_station_code'] = small['end_station_code'].str.replace(')', '')

In [15]:
small['end_station'] = small['end_station'].str.strip()

In [16]:
small.head()

,Start date,End date,Start station,End station,Member Type,start_date_short,end_date_short,time_diff,season,mnth,...,workingday,weathersit,temp,atemp,hum,windspeed,start_station,start_station_code,end_station,end_station_code
0,2011-06-30 23:59:00,2011-07-01 00:09:00,11th & Kenyon St NW (31102),Calvert St & Woodley Pl NW (31106),Registered,2011-06-30,2011-07-01,10.0,3,6,...,1,1,0.696667,0.637008,0.434167,0.185312,11th & Kenyon St NW,31102,Calvert St & Woodley Pl NW,31106
1,2011-06-30 23:58:00,2011-07-01 00:03:00,Lamont & Mt Pleasant NW (31107),Adams Mill & Columbia Rd NW (31104),Registered,2011-06-30,2011-07-01,5.0,3,6,...,1,1,0.696667,0.637008,0.434167,0.185312,Lamont & Mt Pleasant NW,31107,Adams Mill & Columbia Rd NW,31104
2,2011-06-30 23:59:00,2011-07-01 00:09:00,Massachusetts Ave & Dupont Circle NW (31200),Adams Mill & Columbia Rd NW (31104),Casual,2011-06-30,2011-07-01,10.0,3,6,...,1,1,0.696667,0.637008,0.434167,0.185312,Massachusetts Ave & Dupont Circle NW,31200,Adams Mill & Columbia Rd NW,31104
3,2011-06-30 23:58:00,2011-07-01 00:17:00,4th St & Massachusetts Ave NW (31604),C & O Canal & Wisconsin Ave NW (31225),Registered,2011-06-30,2011-07-01,19.0,3,6,...,1,1,0.696667,0.637008,0.434167,0.185312,4th St & Massachusetts Ave NW,31604,C & O Canal & Wisconsin Ave NW,31225
4,2011-06-30 23:58:00,2011-07-01 00:28:00,10th St & Constitution Ave NW (31219),14th & D St NW / Ronald Reagan Building (31231),Casual,2011-06-30,2011-07-01,30.0,3,6,...,1,1,0.696667,0.637008,0.434167,0.185312,10th St & Constitution Ave NW,31219,14th & D St NW / Ronald Reagan Building,31231


In [17]:
columns = list(small.columns)
columns

['Start date',
 'End date',
 'Start station',
 'End station',
 'Member Type',
 'start_date_short',
 'end_date_short',
 'time_diff',
 'season',
 'mnth',
 'holiday',
 'weekday',
 'workingday',
 'weathersit',
 'temp',
 'atemp',
 'hum',
 'windspeed',
 'start_station',
 'start_station_code',
 'end_station',
 'end_station_code']

In [18]:
list_cols = ['Start date',
 'End date',
 'start_station',
 'start_station_code',
 'end_station',
 'end_station_code',
 'Member Type',
 'start_date_short',
 'end_date_short',
 'time_diff',
 'season',
 'mnth',
 'holiday',
 'weekday',
 'workingday',
 'weathersit',
 'temp',
 'atemp',
 'hum',
 'windspeed']

In [19]:
small = small[list_cols]

In [20]:
small.shape

(1394542, 20)

In [21]:
merged_data = pd.merge(small, lat_lon, how='left', left_on='start_station', right_on='ADDRESS')

In [22]:
merged_data.shape

(1394542, 23)

In [23]:
merged_data.head()

,Start date,End date,start_station,start_station_code,end_station,end_station_code,Member Type,start_date_short,end_date_short,time_diff,...,weekday,workingday,weathersit,temp,atemp,hum,windspeed,ADDRESS,LATITUDE,LONGITUDE
0,2011-06-30 23:59:00,2011-07-01 00:09:00,11th & Kenyon St NW,31102,Calvert St & Woodley Pl NW,31106,Registered,2011-06-30,2011-07-01,10.0,...,4,1,1,0.696667,0.637008,0.434167,0.185312,11th & Kenyon St NW,38.929464,-77.027822
1,2011-06-30 23:58:00,2011-07-01 00:03:00,Lamont & Mt Pleasant NW,31107,Adams Mill & Columbia Rd NW,31104,Registered,2011-06-30,2011-07-01,5.0,...,4,1,1,0.696667,0.637008,0.434167,0.185312,Lamont & Mt Pleasant NW,38.931900,-77.038800
2,2011-06-30 23:59:00,2011-07-01 00:09:00,Massachusetts Ave & Dupont Circle NW,31200,Adams Mill & Columbia Rd NW,31104,Casual,2011-06-30,2011-07-01,10.0,...,4,1,1,0.696667,0.637008,0.434167,0.185312,Massachusetts Ave & Dupont Circle NW,38.910100,-77.044400
3,2011-06-30 23:58:00,2011-07-01 00:17:00,4th St & Massachusetts Ave NW,31604,C & O Canal & Wisconsin Ave NW,31225,Registered,2011-06-30,2011-07-01,19.0,...,4,1,1,0.696667,0.637008,0.434167,0.185312,NaN,NaN,NaN
4,2011-06-30 23:58:00,2011-07-01 00:28:00,10th St & Constitution Ave NW,31219,14th & D St NW / Ronald Reagan Building,31231,Casual,2011-06-30,2011-07-01,30.0,...,4,1,1,0.696667,0.637008,0.434167,0.185312,10th St & Constitution Ave NW,38.893028,-77.026013


In [24]:
merged_data = merged_data[merged_data['LATITUDE'].notnull()]

In [25]:
sum(merged_data['LONGITUDE'].isnull())

0

In [26]:
merged_data = merged_data.rename(columns={'LATITUDE': 'start_lat', 'LONGITUDE': 'start_lon'})

In [27]:
merged_data.head()

,Start date,End date,start_station,start_station_code,end_station,end_station_code,Member Type,start_date_short,end_date_short,time_diff,...,weekday,workingday,weathersit,temp,atemp,hum,windspeed,ADDRESS,start_lat,start_lon
0,2011-06-30 23:59:00,2011-07-01 00:09:00,11th & Kenyon St NW,31102,Calvert St & Woodley Pl NW,31106,Registered,2011-06-30,2011-07-01,10.0,...,4,1,1,0.696667,0.637008,0.434167,0.185312,11th & Kenyon St NW,38.929464,-77.027822
1,2011-06-30 23:58:00,2011-07-01 00:03:00,Lamont & Mt Pleasant NW,31107,Adams Mill & Columbia Rd NW,31104,Registered,2011-06-30,2011-07-01,5.0,...,4,1,1,0.696667,0.637008,0.434167,0.185312,Lamont & Mt Pleasant NW,38.931900,-77.038800
2,2011-06-30 23:59:00,2011-07-01 00:09:00,Massachusetts Ave & Dupont Circle NW,31200,Adams Mill & Columbia Rd NW,31104,Casual,2011-06-30,2011-07-01,10.0,...,4,1,1,0.696667,0.637008,0.434167,0.185312,Massachusetts Ave & Dupont Circle NW,38.910100,-77.044400
4,2011-06-30 23:58:00,2011-07-01 00:28:00,10th St & Constitution Ave NW,31219,14th & D St NW / Ronald Reagan Building,31231,Casual,2011-06-30,2011-07-01,30.0,...,4,1,1,0.696667,0.637008,0.434167,0.185312,10th St & Constitution Ave NW,38.893028,-77.026013
6,2011-06-30 23:58:00,2011-07-01 00:28:00,10th St & Constitution Ave NW,31219,14th & D St NW / Ronald Reagan Building,31231,Casual,2011-06-30,2011-07-01,30.0,...,4,1,1,0.696667,0.637008,0.434167,0.185312,10th St & Constitution Ave NW,38.893028,-77.026013


In [28]:
full_merge = pd.merge(merged_data, lat_lon, how='left', left_on='end_station', right_on='ADDRESS')

In [29]:
full_merge.head()

,Start date,End date,start_station,start_station_code,end_station,end_station_code,Member Type,start_date_short,end_date_short,time_diff,...,temp,atemp,hum,windspeed,ADDRESS_x,start_lat,start_lon,ADDRESS_y,LATITUDE,LONGITUDE
0,2011-06-30 23:59:00,2011-07-01 00:09:00,11th & Kenyon St NW,31102,Calvert St & Woodley Pl NW,31106,Registered,2011-06-30,2011-07-01,10.0,...,0.696667,0.637008,0.434167,0.185312,11th & Kenyon St NW,38.929464,-77.027822,Calvert St & Woodley Pl NW,38.923583,-77.050046
1,2011-06-30 23:58:00,2011-07-01 00:03:00,Lamont & Mt Pleasant NW,31107,Adams Mill & Columbia Rd NW,31104,Registered,2011-06-30,2011-07-01,5.0,...,0.696667,0.637008,0.434167,0.185312,Lamont & Mt Pleasant NW,38.931900,-77.038800,Adams Mill & Columbia Rd NW,38.922925,-77.042581
2,2011-06-30 23:59:00,2011-07-01 00:09:00,Massachusetts Ave & Dupont Circle NW,31200,Adams Mill & Columbia Rd NW,31104,Casual,2011-06-30,2011-07-01,10.0,...,0.696667,0.637008,0.434167,0.185312,Massachusetts Ave & Dupont Circle NW,38.910100,-77.044400,Adams Mill & Columbia Rd NW,38.922925,-77.042581
3,2011-06-30 23:58:00,2011-07-01 00:28:00,10th St & Constitution Ave NW,31219,14th & D St NW / Ronald Reagan Building,31231,Casual,2011-06-30,2011-07-01,30.0,...,0.696667,0.637008,0.434167,0.185312,10th St & Constitution Ave NW,38.893028,-77.026013,14th & D St NW / Ronald Reagan Building,38.894514,-77.031617
4,2011-06-30 23:58:00,2011-07-01 00:28:00,10th St & Constitution Ave NW,31219,14th & D St NW / Ronald Reagan Building,31231,Casual,2011-06-30,2011-07-01,30.0,...,0.696667,0.637008,0.434167,0.185312,10th St & Constitution Ave NW,38.893028,-77.026013,14th & D St NW / Ronald Reagan Building,38.894514,-77.031617


In [30]:
sum(full_merge['LONGITUDE'].isnull())

132379

In [31]:
full_merge = full_merge[full_merge['LATITUDE'].notnull()]

In [32]:
full_merge.shape

(1103128, 26)

In [33]:
full_merge = full_merge.rename(columns={'LATITUDE': 'end_lat', 'LONGITUDE': 'end_lon'})

In [34]:
full_merge.head()

,Start date,End date,start_station,start_station_code,end_station,end_station_code,Member Type,start_date_short,end_date_short,time_diff,...,temp,atemp,hum,windspeed,ADDRESS_x,start_lat,start_lon,ADDRESS_y,end_lat,end_lon
0,2011-06-30 23:59:00,2011-07-01 00:09:00,11th & Kenyon St NW,31102,Calvert St & Woodley Pl NW,31106,Registered,2011-06-30,2011-07-01,10.0,...,0.696667,0.637008,0.434167,0.185312,11th & Kenyon St NW,38.929464,-77.027822,Calvert St & Woodley Pl NW,38.923583,-77.050046
1,2011-06-30 23:58:00,2011-07-01 00:03:00,Lamont & Mt Pleasant NW,31107,Adams Mill & Columbia Rd NW,31104,Registered,2011-06-30,2011-07-01,5.0,...,0.696667,0.637008,0.434167,0.185312,Lamont & Mt Pleasant NW,38.931900,-77.038800,Adams Mill & Columbia Rd NW,38.922925,-77.042581
2,2011-06-30 23:59:00,2011-07-01 00:09:00,Massachusetts Ave & Dupont Circle NW,31200,Adams Mill & Columbia Rd NW,31104,Casual,2011-06-30,2011-07-01,10.0,...,0.696667,0.637008,0.434167,0.185312,Massachusetts Ave & Dupont Circle NW,38.910100,-77.044400,Adams Mill & Columbia Rd NW,38.922925,-77.042581
3,2011-06-30 23:58:00,2011-07-01 00:28:00,10th St & Constitution Ave NW,31219,14th & D St NW / Ronald Reagan Building,31231,Casual,2011-06-30,2011-07-01,30.0,...,0.696667,0.637008,0.434167,0.185312,10th St & Constitution Ave NW,38.893028,-77.026013,14th & D St NW / Ronald Reagan Building,38.894514,-77.031617
4,2011-06-30 23:58:00,2011-07-01 00:28:00,10th St & Constitution Ave NW,31219,14th & D St NW / Ronald Reagan Building,31231,Casual,2011-06-30,2011-07-01,30.0,...,0.696667,0.637008,0.434167,0.185312,10th St & Constitution Ave NW,38.893028,-77.026013,14th & D St NW / Ronald Reagan Building,38.894514,-77.031617


In [35]:
list(full_merge.columns)

['Start date',
 'End date',
 'start_station',
 'start_station_code',
 'end_station',
 'end_station_code',
 'Member Type',
 'start_date_short',
 'end_date_short',
 'time_diff',
 'season',
 'mnth',
 'holiday',
 'weekday',
 'workingday',
 'weathersit',
 'temp',
 'atemp',
 'hum',
 'windspeed',
 'ADDRESS_x',
 'start_lat',
 'start_lon',
 'ADDRESS_y',
 'end_lat',
 'end_lon']

In [36]:
new_list = ['Start date',
 'End date',
 'start_station',
 'start_station_code',
 'end_station',
 'end_station_code',
 'Member Type',
 'start_date_short',
 'end_date_short',
 'time_diff',
 'season',
 'mnth',
 'holiday',
 'weekday',
 'workingday',
 'weathersit',
 'temp',
 'atemp',
 'hum',
 'windspeed',
 'start_lat',
 'start_lon',
 'end_lat',
 'end_lon']

In [37]:
full_merge = full_merge[new_list]

In [38]:
full_merge['lat_lon_start'] = list(zip(full_merge.start_lat, full_merge.start_lon))

In [39]:
full_merge['lat_lon_end'] = list(zip(full_merge.end_lat, full_merge.end_lon))

In [40]:
full_merge.head()

,Start date,End date,start_station,start_station_code,end_station,end_station_code,Member Type,start_date_short,end_date_short,time_diff,...,temp,atemp,hum,windspeed,start_lat,start_lon,end_lat,end_lon,lat_lon_start,lat_lon_end
0,2011-06-30 23:59:00,2011-07-01 00:09:00,11th & Kenyon St NW,31102,Calvert St & Woodley Pl NW,31106,Registered,2011-06-30,2011-07-01,10.0,...,0.696667,0.637008,0.434167,0.185312,38.929464,-77.027822,38.923583,-77.050046,"(38.929464, -77.027822)","(38.923583, -77.050046)"
1,2011-06-30 23:58:00,2011-07-01 00:03:00,Lamont & Mt Pleasant NW,31107,Adams Mill & Columbia Rd NW,31104,Registered,2011-06-30,2011-07-01,5.0,...,0.696667,0.637008,0.434167,0.185312,38.931900,-77.038800,38.922925,-77.042581,"(38.9319, -77.0388)","(38.922925, -77.042581)"
2,2011-06-30 23:59:00,2011-07-01 00:09:00,Massachusetts Ave & Dupont Circle NW,31200,Adams Mill & Columbia Rd NW,31104,Casual,2011-06-30,2011-07-01,10.0,...,0.696667,0.637008,0.434167,0.185312,38.910100,-77.044400,38.922925,-77.042581,"(38.9101, -77.0444)","(38.922925, -77.042581)"
3,2011-06-30 23:58:00,2011-07-01 00:28:00,10th St & Constitution Ave NW,31219,14th & D St NW / Ronald Reagan Building,31231,Casual,2011-06-30,2011-07-01,30.0,...,0.696667,0.637008,0.434167,0.185312,38.893028,-77.026013,38.894514,-77.031617,"(38.893028, -77.026013)","(38.894514, -77.031617)"
4,2011-06-30 23:58:00,2011-07-01 00:28:00,10th St & Constitution Ave NW,31219,14th & D St NW / Ronald Reagan Building,31231,Casual,2011-06-30,2011-07-01,30.0,...,0.696667,0.637008,0.434167,0.185312,38.893028,-77.026013,38.894514,-77.031617,"(38.893028, -77.026013)","(38.894514, -77.031617)"


In [41]:
full_merge['miles'] = full_merge.apply(lambda row: vincenty(row['lat_lon_start'], row['lat_lon_end']).miles, axis=1)

In [42]:
full_merge.head()

,Start date,End date,start_station,start_station_code,end_station,end_station_code,Member Type,start_date_short,end_date_short,time_diff,...,atemp,hum,windspeed,start_lat,start_lon,end_lat,end_lon,lat_lon_start,lat_lon_end,miles
0,2011-06-30 23:59:00,2011-07-01 00:09:00,11th & Kenyon St NW,31102,Calvert St & Woodley Pl NW,31106,Registered,2011-06-30,2011-07-01,10.0,...,0.637008,0.434167,0.185312,38.929464,-77.027822,38.923583,-77.050046,"(38.929464, -77.027822)","(38.923583, -77.050046)",1.264341
1,2011-06-30 23:58:00,2011-07-01 00:03:00,Lamont & Mt Pleasant NW,31107,Adams Mill & Columbia Rd NW,31104,Registered,2011-06-30,2011-07-01,5.0,...,0.637008,0.434167,0.185312,38.931900,-77.038800,38.922925,-77.042581,"(38.9319, -77.0388)","(38.922925, -77.042581)",0.651763
2,2011-06-30 23:59:00,2011-07-01 00:09:00,Massachusetts Ave & Dupont Circle NW,31200,Adams Mill & Columbia Rd NW,31104,Casual,2011-06-30,2011-07-01,10.0,...,0.637008,0.434167,0.185312,38.910100,-77.044400,38.922925,-77.042581,"(38.9101, -77.0444)","(38.922925, -77.042581)",0.890094
3,2011-06-30 23:58:00,2011-07-01 00:28:00,10th St & Constitution Ave NW,31219,14th & D St NW / Ronald Reagan Building,31231,Casual,2011-06-30,2011-07-01,30.0,...,0.637008,0.434167,0.185312,38.893028,-77.026013,38.894514,-77.031617,"(38.893028, -77.026013)","(38.894514, -77.031617)",0.319015
4,2011-06-30 23:58:00,2011-07-01 00:28:00,10th St & Constitution Ave NW,31219,14th & D St NW / Ronald Reagan Building,31231,Casual,2011-06-30,2011-07-01,30.0,...,0.637008,0.434167,0.185312,38.893028,-77.026013,38.894514,-77.031617,"(38.893028, -77.026013)","(38.894514, -77.031617)",0.319015


In [43]:
full_merge.shape

(1103128, 27)

In [44]:
full_merge.to_csv('bikeshare_lat_lon.csv')

In [45]:
full_merge.drop('lat_lon_start', 1, inplace=True)

In [46]:
full_merge.drop('lat_lon_end', 1, inplace=True)

In [47]:
full_merge.head()

,Start date,End date,start_station,start_station_code,end_station,end_station_code,Member Type,start_date_short,end_date_short,time_diff,...,weathersit,temp,atemp,hum,windspeed,start_lat,start_lon,end_lat,end_lon,miles
0,2011-06-30 23:59:00,2011-07-01 00:09:00,11th & Kenyon St NW,31102,Calvert St & Woodley Pl NW,31106,Registered,2011-06-30,2011-07-01,10.0,...,1,0.696667,0.637008,0.434167,0.185312,38.929464,-77.027822,38.923583,-77.050046,1.264341
1,2011-06-30 23:58:00,2011-07-01 00:03:00,Lamont & Mt Pleasant NW,31107,Adams Mill & Columbia Rd NW,31104,Registered,2011-06-30,2011-07-01,5.0,...,1,0.696667,0.637008,0.434167,0.185312,38.931900,-77.038800,38.922925,-77.042581,0.651763
2,2011-06-30 23:59:00,2011-07-01 00:09:00,Massachusetts Ave & Dupont Circle NW,31200,Adams Mill & Columbia Rd NW,31104,Casual,2011-06-30,2011-07-01,10.0,...,1,0.696667,0.637008,0.434167,0.185312,38.910100,-77.044400,38.922925,-77.042581,0.890094
3,2011-06-30 23:58:00,2011-07-01 00:28:00,10th St & Constitution Ave NW,31219,14th & D St NW / Ronald Reagan Building,31231,Casual,2011-06-30,2011-07-01,30.0,...,1,0.696667,0.637008,0.434167,0.185312,38.893028,-77.026013,38.894514,-77.031617,0.319015
4,2011-06-30 23:58:00,2011-07-01 00:28:00,10th St & Constitution Ave NW,31219,14th & D St NW / Ronald Reagan Building,31231,Casual,2011-06-30,2011-07-01,30.0,...,1,0.696667,0.637008,0.434167,0.185312,38.893028,-77.026013,38.894514,-77.031617,0.319015


In [49]:
full_merge.to_csv('smaller_bikeshare_lat_lon.csv')

In [50]:
full_merge.drop('atemp', 1, inplace=True)

In [51]:
full_merge.to_csv('less_bikeshare_lat_lon.csv')